<a href="https://colab.research.google.com/github/LorenaGiVa/Caso---Marketing/blob/main/b_exploracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import plotly.graph_objs as go
import plotly.express as px


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
##### conectarse a BD #######
conn= sql.connect('/content/drive/MyDrive/Caso marketing analítica/data/db_movies')
cur=conn.cursor()

In [6]:
### para ver las tablas que hay en la base de datos
cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()

[('ratings',), ('movies',)]

In [7]:
########### traer tabla de BD a python ####
movies= pd.read_sql("select *  from movies", conn)
ratings = pd.read_sql('select * from ratings', conn)

In [8]:
### ver el tipo de datos y faltantes
print(movies.info())
print(ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None


No hay nulos

In [9]:
### datos duplicados
print(movies.duplicated().sum())
print(ratings.duplicated().sum())

0
0


In [10]:
# distribuciin de las calificaciones
cr=pd.read_sql(""" select rating,
                    count(*) as conteo
                    from ratings
                    group by rating
                    order by conteo desc""", conn)

In [11]:
data  = go.Bar( x=cr.rating,y=cr.conteo, text=cr.conteo, textposition="outside")
Layout=go.Layout(title="Count of ratings",xaxis={'title':'Rating'},yaxis={'title':'Count'})
go.Figure(data,Layout)

In [13]:
### calcular cada usuario cuantas peliculas calificó
rating_users = pd.read_sql(''' select userId,
                                count(*) as cnt_rat
                                from ratings
                                group by userId
                                order by cnt_rat asc
                                ''',conn )

In [17]:
rating_users = pd.read_sql(''' select userId,
                                count(*) as cnt_rat
                                from ratings
                                group by userId
                                order by cnt_rat asc
                                ''',conn )

In [18]:
rating_users

,userId,cnt_rat
0,53,20
1,147,20
2,189,20
3,194,20
4,207,20
...,...,...
605,274,1346
606,448,1864
607,474,2108
608,599,2478


In [14]:
fig  = px.histogram(rating_users, x= 'cnt_rat', title= 'Histograma frecuencia de número de calificaciones por usuario')
fig.show()

In [19]:
rating_users.describe()

,userId,cnt_rat
count,610.000000,610.000000
mean,305.500000,165.304918
std,176.236111,269.480584
min,1.000000,20.000000
25%,153.250000,35.000000
50%,305.500000,70.500000
75%,457.750000,168.000000
max,610.000000,2698.000000


In [20]:
rating_users2 = pd.read_sql('''select userId,
                                count(*) as cnt_rat
                                from ratings
                                group by userId
                                having cnt_rat <=1000
                                order by cnt_rat asc
                                ''',conn )

In [21]:
fig  = px.histogram(rating_users2, x= 'cnt_rat', title= 'Histograma frecuencia de número de calificaciones por usuario')
fig.show()

In [22]:
fig  = px.histogram(rating_users2, x= 'cnt_rat', title= 'Histograma frecuencia de número de calificaciones por usuario')
fig.show()

,userId,cnt_rat
count,598.000000,598.000000
mean,303.608696,137.657191
std,176.083249,169.447554
min,1.000000,20.000000
25%,151.250000,35.000000
50%,303.500000,69.000000
75%,456.750000,162.500000
max,609.000000,977.000000


In [28]:
### calificacion de cada pelicula
rating_movie = pd.read_sql('''select movieId,
                                count(*) as cnt_rat
                                from ratings
                                group by movieId
                                order by cnt_rat desc
                                ''',conn )


In [29]:
fig  = px.histogram(rating_movie, x= 'cnt_rat', title= 'Histograma frecuencia de número de calificaciones por pelicula')
fig.show()

In [31]:
rating_movie.describe()

,movieId,cnt_rat
count,9724.000000,9724.000000
mean,42245.024373,10.369807
std,52191.137320,22.401005
min,1.000000,1.000000
25%,3245.500000,1.000000
50%,7300.000000,3.000000
75%,76739.250000,9.000000
max,193609.000000,329.000000


In [32]:
####peliculas que tengan más de 5 calificaciones
rating_movie2=pd.read_sql(''' select movieId,
                                count(*) as cnt_rat
                                from ratings
                                group by movieId
                                having cnt_rat >= 5
                                order by cnt_rat desc
                                ''',conn )

In [33]:
fig  = px.histogram(rating_movie2, x= 'cnt_rat', title= 'Histograma frecuencia de número de calificaciones por pelicula')
fig.show()

In [34]:
rating_movie2.describe()

,movieId,cnt_rat
count,3650.000000,3650.000000
mean,23839.433151,24.732603
std,38394.748531,31.703576
min,1.000000,5.000000
25%,1784.250000,7.000000
50%,3866.000000,13.000000
75%,36664.750000,28.000000
max,187595.000000,329.000000
